# Jupyter Notebook Para DATA MINING

In [23]:
# Importações
import pandas
import sqlite3

# Carregando os dados
data = pandas.read_csv('data/acidentes2023_todas_causas_tipos.csv', encoding='ISO-8859-1', sep=";")
data.info()

# Flags para não rodar o mesmo código 2 vezes, para forçar isso adicionar o código "SECURITY_FLAGS['A_FLAG_ESPECÍFICA'] = False" antes da validação de flag
SECURITY_FLAGS = {}
def should_run(flag):
    response = flag not in SECURITY_FLAGS or not SECURITY_FLAGS[flag]
    SECURITY_FLAGS[flag] = True
    return response

def snapshot(table_name):
    pass
    # con = sqlite3.connect('data/acidentes.sqlite')
    # data.to_sql(name=table_name, con=con, if_exists='replace', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514799 entries, 0 to 514798
Data columns (total 37 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      514799 non-null  int64  
 1   pesid                   514799 non-null  int64  
 2   data_inversa            514799 non-null  object 
 3   dia_semana              514799 non-null  object 
 4   horario                 514799 non-null  object 
 5   uf                      514799 non-null  object 
 6   br                      513037 non-null  float64
 7   km                      513037 non-null  object 
 8   municipio               514799 non-null  object 
 9   causa_principal         514799 non-null  object 
 10  causa_acidente          514799 non-null  object 
 11  ordem_tipo_acidente     514799 non-null  int64  
 12  tipo_acidente           514799 non-null  object 
 13  classificacao_acidente  514799 non-null  object 
 14  fase_dia            

In [24]:
# Subir base suja para SQLite para visualizar e pesquisar
if should_run('SQLite - Dirty data'):
    snapshot('dirty_2023_data')

In [25]:
for column in data.columns:
    unique = data[column].unique()
    unique_len = len(unique)

    if unique_len > 50:
        print(column, unique_len)
        continue

    print(column, unique_len, unique)

id 67724
pesid 166836
data_inversa 365
dia_semana 7 ['domingo' 'segunda-feira' 'terça-feira' 'quarta-feira' 'quinta-feira'
 'sexta-feira' 'sábado']
horario 1406
uf 27 ['MG' 'RS' 'SE' 'PR' 'SP' 'ES' 'SC' 'RJ' 'MT' 'PE' 'PA' 'CE' 'AM' 'BA'
 'DF' 'RO' 'RN' 'GO' 'PB' 'AL' 'MS' 'PI' 'TO' 'MA' 'AP' 'AC' 'RR']
br 119
km 7638
municipio 1825
causa_principal 2 ['Sim' 'Não']
causa_acidente 76
ordem_tipo_acidente 12 [ 2  1  4  3  5  6  7  9  8 10 11 12]
tipo_acidente 16 ['Queda de ocupante de veículo' 'Tombamento' 'Capotamento'
 'Saída de leito carroçável' 'Colisão com objeto' 'Colisão traseira'
 'Atropelamento de Pedestre' 'Colisão transversal'
 'Atropelamento de Animal' 'Colisão frontal'
 'Colisão lateral mesmo sentido' 'Colisão lateral sentido oposto'
 'Derramamento de carga' 'Engavetamento' 'Incêndio' 'Eventos atípicos']
classificacao_acidente 3 ['Com Vítimas Feridas' 'Com Vítimas Fatais' 'Sem Vítimas']
fase_dia 4 ['Plena Noite' 'Pleno dia' 'Amanhecer' 'Anoitecer']
sentido_via 3 ['Crescente' '

In [26]:
data.isnull().sum()

id                            0
pesid                         0
data_inversa                  0
dia_semana                    0
horario                       0
uf                            0
br                         1762
km                         1762
municipio                     0
causa_principal               0
causa_acidente                0
ordem_tipo_acidente           0
tipo_acidente                 0
classificacao_acidente        0
fase_dia                      0
sentido_via                   0
condicao_metereologica        0
tipo_pista                    0
tracado_via                   0
uso_solo                      0
id_veiculo                    0
tipo_veiculo                  0
marca                     21014
ano_fabricacao_veiculo    27439
tipo_envolvido                0
estado_fisico                 0
idade                     49369
sexo                          0
ilesos                        0
feridos_leves                 0
feridos_graves                0
mortos  

In [27]:
print('Number of duplicate rows = %d' % (data.duplicated().sum()))

Number of duplicate rows = 0


# Conversão de dados

In [28]:
if should_run('week_2_number'):
    # Dia da semana
    week_2_number = {
        'segunda-feira': 0,
        'terça-feira': 1,
        'quarta-feira': 2,
        'quinta-feira': 3,
        'sexta-feira': 4,
        'sábado': 5,
        'domingo': 6,
    }
    
    for key in week_2_number.keys():
        data['dia_semana'].replace(key, week_2_number[key], inplace=True)
    
    data['dia_semana'].unique()

/tmp/ipykernel_42337/694074818.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['dia_semana'].replace(key, week_2_number[key], inplace=True)
/tmp/ipykernel_42337/694074818.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['dia_semana'].replace(key, week_2_number[key], inplace=True)


In [29]:
if should_run('data_hora'):
    data_hora = []
    for i in range(len(data)):
        data_hora.append(data['data_inversa'][i] + ' ' + data['horario'][i][:5] + 'Z')
    
    data['data_hora'] = pandas.to_datetime(data_hora, format='ISO8601')


## Cast para str

In [30]:
if should_run('fields_2_str'):
    fields_2_str = ['data_inversa', 'horario', 'uf', 'municipio', 'causa_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'tipo_veiculo', 'marca', 'tipo_envolvido', 'estado_fisico', 'sexo', 'latitude', 'longitude', 'regional', 'delegacia', 'uop']
    for field in fields_2_str:
        data[field] = data[field].convert_dtypes('string')


## Cast Boolean

In [31]:
if should_run('causa_principal'):
    data['causa_principal'] = data['causa_principal'].replace(['Sim', 'sim'], True)
    data['causa_principal'] = data['causa_principal'].replace(['Não', 'não'], False)

if should_run('uso_solo'):
    data['uso_solo'] = data['uso_solo'].replace(['Sim', 'sim'], True)
    data['uso_solo'] = data['uso_solo'].replace(['Não', 'não'], False)

/tmp/ipykernel_42337/1626753868.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['causa_principal'] = data['causa_principal'].replace(['Não', 'não'], False)
/tmp/ipykernel_42337/1626753868.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['uso_solo'] = data['uso_solo'].replace(['Não', 'não'], False)


## Cast Float

In [32]:
if should_run('km_to_float'):
    km_to_float = []
    for value in data['km']:
        tmp = value
        if isinstance(value, str):
            tmp = value.replace(',', '.')
    
        km_to_float.append(float(tmp))
    
    data['km'] = km_to_float

## Reordenação

In [33]:
if should_run('sort'):
    data.columns.tolist()
    cols = ['id', 'pesid', 'data_hora', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km', 'municipio', 'causa_principal', 'causa_acidente', 'ordem_tipo_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo', 'id_veiculo', 'tipo_veiculo', 'marca', 'ano_fabricacao_veiculo', 'tipo_envolvido', 'estado_fisico', 'idade', 'sexo', 'ilesos', 'feridos_leves', 'feridos_graves', 'mortos', 'latitude', 'longitude', 'regional', 'delegacia', 'uop',]
    data = data[cols]

## Tratativa do Traçado da Via (TAGS)

In [34]:
def cast_tags(field, separator = ';'):
    uniques = {}
    new_tags = []
    for x in data[field]:
        tmp = x.strip().split(separator)
        valid_tags = []
        for index, y in enumerate(tmp):
            if y not in uniques:
                # Como o campo original só aceita 30 chars o último pode ser cortado,
                # Caso ele não tenha sido colocado no dicionário, pode cair no caso de uma Tag cortada
                if len(tmp) <= 1 or index != len(tmp) - 1:
                    valid_tags.append(y)
                    uniques[y] = 1
                continue
    
            valid_tags.append(y)
            uniques[y] += 1
    
        new_tags.append(separator.join(valid_tags))

    print(uniques)
    return new_tags

# print([x.strip() for x in data['tracado_via'].unique()])
if should_run('tracado_via'):
    data['tracado_via'] = cast_tags('tracado_via')


{'Reta': 377638, 'Curva': 94743, 'Em Obras': 14390, 'Declive': 72329, 'Aclive': 47151, 'Retorno Regulamentado': 10708, 'Rotatória': 9667, 'Viaduto': 7192, 'Interseção de Vias': 35125, 'Ponte': 7012, 'Desvio Temporário': 2090, 'Túnel': 185}


# Pós-processamento

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514799 entries, 0 to 514798
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   id                      514799 non-null  int64              
 1   pesid                   514799 non-null  int64              
 2   data_hora               514799 non-null  datetime64[ns, UTC]
 3   data_inversa            514799 non-null  string             
 4   dia_semana              514799 non-null  int64              
 5   horario                 514799 non-null  string             
 6   uf                      514799 non-null  string             
 7   br                      513037 non-null  float64            
 8   km                      513037 non-null  float64            
 9   municipio               514799 non-null  string             
 10  causa_principal         514799 non-null  bool               
 11  causa_acidente          51

In [36]:
data

,id,pesid,data_hora,data_inversa,dia_semana,horario,uf,br,km,municipio,...,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop
0,496506,1082142,2023-01-01 00:15:00+00:00,2023-01-01,6,00:15:00,MG,116.0,587.0,MANHUACU,...,Masculino,0,1,0,0,"-20,24173903","-42,15868042",SPRF-MG,DEL06-MG,UOP03-DEL06-MG
1,496506,1082142,2023-01-01 00:15:00+00:00,2023-01-01,6,00:15:00,MG,116.0,587.0,MANHUACU,...,Masculino,0,1,0,0,"-20,24173903","-42,15868042",SPRF-MG,DEL06-MG,UOP03-DEL06-MG
2,496506,1082142,2023-01-01 00:15:00+00:00,2023-01-01,6,00:15:00,MG,116.0,587.0,MANHUACU,...,Masculino,0,1,0,0,"-20,24173903","-42,15868042",SPRF-MG,DEL06-MG,UOP03-DEL06-MG
3,496506,1082142,2023-01-01 00:15:00+00:00,2023-01-01,6,00:15:00,MG,116.0,587.0,MANHUACU,...,Masculino,0,1,0,0,"-20,24173903","-42,15868042",SPRF-MG,DEL06-MG,UOP03-DEL06-MG
4,496507,1082138,2023-01-01 00:20:00+00:00,2023-01-01,6,00:20:00,MG,381.0,686.5,LAVRAS,...,Masculino,0,1,0,0,"-21,17928973","-45,12256622",SPRF-MG,DEL04-MG,UOP02-DEL04-MG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514794,589495,1312506,2023-09-10 21:00:00+00:00,2023-09-10,6,21:00:00,SC,101.0,73.3,ARAQUARI,...,Masculino,1,0,0,0,"-26,53567108","-48,71326731",SPRF-SC,DEL03-SC,UOP02-DEL03-SC
514795,589495,1312504,2023-09-10 21:00:00+00:00,2023-09-10,6,21:00:00,SC,101.0,73.3,ARAQUARI,...,Masculino,1,0,0,0,"-26,53567108","-48,71326731",SPRF-SC,DEL03-SC,UOP02-DEL03-SC
514796,589495,1312503,2023-09-10 21:00:00+00:00,2023-09-10,6,21:00:00,SC,101.0,73.3,ARAQUARI,...,Masculino,1,0,0,0,"-26,53567108","-48,71326731",SPRF-SC,DEL03-SC,UOP02-DEL03-SC
514797,589495,1312502,2023-09-10 21:00:00+00:00,2023-09-10,6,21:00:00,SC,101.0,73.3,ARAQUARI,...,Não Informado,0,0,0,0,"-26,53567108","-48,71326731",SPRF-SC,DEL03-SC,UOP02-DEL03-SC


In [37]:
print(data.isnull().sum())
data = data.dropna()
print(data.isnull().sum())

id                            0
pesid                         0
data_hora                     0
data_inversa                  0
dia_semana                    0
horario                       0
uf                            0
br                         1762
km                         1762
municipio                     0
causa_principal               0
causa_acidente                0
ordem_tipo_acidente           0
tipo_acidente                 0
classificacao_acidente        0
fase_dia                      0
sentido_via                   0
condicao_metereologica        0
tipo_pista                    0
tracado_via                   0
uso_solo                      0
id_veiculo                    0
tipo_veiculo                  0
marca                     21014
ano_fabricacao_veiculo    27439
tipo_envolvido                0
estado_fisico                 0
idade                     49369
sexo                          0
ilesos                        0
feridos_leves                 0
feridos_

In [38]:
if should_run('SQLite - Clean data'):
    snapshot('clean_2023_data')

# Análise de Dados

In [39]:
import seaborn
discrete_columns = [
    'data_hora', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km', 'municipio', 'causa_principal', 'causa_acidente', 'ordem_tipo_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo', 'tipo_veiculo', 'marca', 'ano_fabricacao_veiculo', 'tipo_envolvido', 'estado_fisico', 'idade', 'sexo', 'ilesos', 'feridos_leves', 'feridos_graves', 'mortos']

colunas_interesse = ['data_hora', 'horario', 'br', 'km', 'causa_acidente', 'tipo_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'tipo_veiculo', 'ano_fabricacao_veiculo', 'tipo_envolvido', 'idade', 'sexo']

for column in data[colunas_interesse].columns:
    unique = data[column].unique()
    unique_len = len(unique)

    if unique_len > 50:
        print(column, unique_len)
        continue

    print(column, unique_len, unique)

# data[discrete_columns].describe()

data_hora 43867
horario 1391
br 116
km 7548
causa_acidente 76
tipo_acidente 16 <StringArray>
[  'Queda de ocupante de veículo',                     'Tombamento',
                    'Capotamento',      'Saída de leito carroçável',
             'Colisão com objeto',               'Colisão traseira',
      'Atropelamento de Pedestre',            'Colisão transversal',
        'Atropelamento de Animal',                'Colisão frontal',
  'Colisão lateral mesmo sentido', 'Colisão lateral sentido oposto',
          'Derramamento de carga',                  'Engavetamento',
                       'Incêndio',               'Eventos atípicos']
Length: 16, dtype: string
fase_dia 4 <StringArray>
['Plena Noite', 'Pleno dia', 'Amanhecer', 'Anoitecer']
Length: 4, dtype: string
sentido_via 2 <StringArray>
['Crescente', 'Decrescente']
Length: 2, dtype: string
condicao_metereologica 10 <StringArray>
[         'Nublado',   'Garoa/Chuvisco',        'Céu Claro',
            'Chuva', 'Nevoeiro/Neblina', 

In [40]:
bins = pandas.qcut(data['idade'],4)
bins.value_counts(sort=False)

idade
(-0.001, 27.0]    114049
(27.0, 38.0]      120734
(38.0, 49.0]      107186
(49.0, 2022.0]    109643
Name: count, dtype: int64

In [41]:
data['estado_fisico'].info()

<class 'pandas.core.series.Series'>
Index: 451612 entries, 0 to 514798
Series name: estado_fisico
Non-Null Count   Dtype 
--------------   ----- 
451612 non-null  string
dtypes: string(1)
memory usage: 6.9 MB


# Model Decision Tree

In [42]:
colunas_interesse = [
    #'data_hora',
    'horario', 'br', 'km', 'causa_acidente', 'tipo_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'tipo_veiculo', 'ano_fabricacao_veiculo', 'tipo_envolvido', 'idade', 'sexo', 'estado_fisico',
]
data_interesse = data[colunas_interesse]

 
from sklearn.preprocessing import LabelEncoder
# Convert categorical variables to numerical values
le = LabelEncoder()

data_interesse_le = data_interesse[['br', 'km', 'ano_fabricacao_veiculo', 'idade', 'estado_fisico']]

string_to_le = ['causa_acidente', 'tipo_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tipo_veiculo', 'tipo_envolvido', 'sexo']
for data_column in data_interesse[string_to_le]:
    data_interesse_le[data_column] = le.fit_transform(data_interesse[data_column])


/tmp/ipykernel_42337/93653000.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_interesse_le[data_column] = le.fit_transform(data_interesse[data_column])
/tmp/ipykernel_42337/93653000.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_interesse_le[data_column] = le.fit_transform(data_interesse[data_column])
/tmp/ipykernel_42337/93653000.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [223]:
from sklearn.tree import DecisionTreeClassifier

features = ['br', 'km', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tipo_veiculo', 'ano_fabricacao_veiculo', 'tipo_envolvido', 'idade', 'sexo']
# limit = 100000

def test_decision_tree(features_list: [str]):
    x = data_interesse_le[features_list]#[:limit]
    y = data_interesse_le['estado_fisico']#[:limit]

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
    clf = DecisionTreeClassifier(max_depth=30) # Create a decision tree classifier object
    clf = clf.fit(x_train, y_train) # Train the classifier using the training data
    y_pred = clf.predict(x_test) # Predict the labels of the test data
    
    # print("features:", features_list)
    # print("Accuracy:", accuracy_score(y_test, y_pred))
    # print({
    #     'accuracy': accuracy_score(y_test, y_pred),
    #     'precision': precision_score(y_test, y_pred, average='macro'),
    #     'recall': recall_score(y_test, y_pred, average='macro'),
    #     'f1_score': f1_score(y_test, y_pred, average='macro')
    # })
    print(f"{accuracy_score(y_test, y_pred)};{precision_score(y_test, y_pred, average='macro')};{recall_score(y_test, y_pred, average='macro')};{f1_score(y_test, y_pred, average='macro')}".replace('.', ','))
    print("")

for i in range(len(features)):
    print(f"without {features[i]}")
    if i == 0:
        test_decision_tree(features[1:len(features)])
    elif i == len(features) - 1:
        test_decision_tree(features[:len(features)-1])
    else:
        test_decision_tree(features[0:i] + features[i+1: len(features)])

test_decision_tree(features)

# text_representation = tree.export_text(clf, max_depth=30)
# print(text_representation)

# import matplotlib.pyplot as plt
# from sklearn import tree
# 
# #Configure the figure size
# plt.figure(figsize=(20, 10), dpi=100)
# 
# # Plot the decision tree
# tree.plot_tree(clf, filled=True, feature_names=features, class_names=['Ileso', 'Lesões Leves', 'Lesões Graves', 'Óbito', 'Não Informado'], max_depth=3)
# 
# # Save the figure as a high-definition image
# plt.savefig('decision_tree.png', dpi=1000)

without br
0,8966298603525139;0,8817742790355678;0,8935714005655349;0,8875070868874315

without km
0,8978920020076171;0,8808011833569316;0,8934379600512091;0,8869525761095578

without fase_dia
0,8964970033361873;0,8824542803042664;0,8872456521110748;0,8848181856942926

without sentido_via
0,9015677127926545;0,8861611911713908;0,896746098923802;0,8913353586416092

without condicao_metereologica
0,90214342653007;0,8883788802171332;0,8972202466842282;0,8927073755014588

without tipo_pista
0,8976041451389094;0,8834626068823243;0,8906306225295395;0,8869675747770867

without tipo_veiculo
0,876037022821883;0,8664807966622586;0,8732798867120739;0,8697971126223912

without ano_fabricacao_veiculo
0,8979289067343745;0,8882842145990413;0,8894897986713453;0,8888587448292214

without tipo_envolvido
0,8967110507513802;0,8828205120624726;0,8919698056300909;0,8872427425827721

without idade
0,8733429777685926;0,8573633378159218;0,8533224237225367;0,8553044140395286

without sexo
0,9032874730595495;0,89

# Model Naive bayes

In [225]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

features = ['br', 'km', 'fase_dia','causa_acidente', 'tipo_acidente', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tipo_veiculo', 'ano_fabricacao_veiculo', 'tipo_envolvido', 'idade', 'sexo']

def test_naive_bayes(features_list: [str]):
    x = data_interesse_le[features_list]
    y = data_interesse_le['estado_fisico']

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

    gnb = GaussianNB().fit(x_train, y_train)    
    y_pred = gnb.predict(x_test)

    print(f"{accuracy_score(y_test, y_pred)};{precision_score(y_test, y_pred, average='macro')};{recall_score(y_test, y_pred, average='macro')};{f1_score(y_test, y_pred, average='macro')}".replace('.', ','))
    print("")


for i in range(len(features)):
    print(f"without {features[i]}")
    if i == 0:
        test_naive_bayes(features[1:len(features)])
    elif i == len(features) - 1:
        test_naive_bayes(features[:len(features)-1])
    else:
        test_naive_bayes(features[0:i] + features[i+1: len(features)])

test_naive_bayes(features)


without br
0,4563121844645862;0,41654155823721595;0,2669174910395062;0,1885214705725365

without km
0,4546440908151516;0,41450813298112865;0,26708500194053525;0,18781842579766808

without fase_dia
0,45603908948658145;0,41783147308892066;0,2668832881358532;0,18878828023204908

without causa_acidente
0,4538690915532461;0,4249979657256962;0,2669357022086395;0,18782795962039744

without tipo_acidente
0,4540240914056272;0,4353774207764459;0,2654240158401467;0,18642958972722784

without sentido_via
0,4543340911103894;0,4450924905724916;0,2657351506502281;0,18654501397593776

without condicao_metereologica
0,4547548049954238;0,4046028258110403;0,2669780680307486;0,19096319584035915

without tipo_pista
0,45459242419769125;0,416621419511076;0,2667629157978167;0,18746859924746492

without tipo_veiculo
0,4547548049954238;0,41636011387029837;0,26650225429179975;0,18641332903762595

without ano_fabricacao_veiculo
0,4708969324791119;0,3824633410699302;0,2807279877870677;0,24231908489833942

without 

In [231]:
from sklearn.neighbors import KNeighborsClassifier

features = ['br', 'km', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tipo_veiculo', 'ano_fabricacao_veiculo', 'tipo_envolvido', 'idade', 'sexo']

def test_knn(features_list: [str]):
    x = data_interesse_le[features_list]
    y = data_interesse_le['estado_fisico']
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

    knn = KNeighborsClassifier(n_neighbors=1, algorithm='auto').fit(x_train, y_train)
    y_pred = knn.predict(x_test)

    print(f"{accuracy_score(y_test, y_pred)};{precision_score(y_test, y_pred, average='macro')};{recall_score(y_test, y_pred, average='macro')};{f1_score(y_test, y_pred, average='macro')}".replace('.', ','))
    print("")

for i in range(len(features)):
    print(f"without {features[i]}")
    if i == 0:
        test_knn(features[1:len(features)])
    elif i == len(features) - 1:
        test_knn(features[:len(features)-1])
    else:
        test_knn(features[0:i] + features[i+1: len(features)])

test_knn(features)

# metrics = {
#     'kNN': {
#         'accuracy': accuracy_score(y_test, y_pred_knn),
#         # 'precision': precision_score(y_test, y_pred_knn, average='macro'),
#         # 'recall': recall_score(y_test, y_pred_knn, average='macro'),
#         # 'f1_score': f1_score(y_test, y_pred_knn, average='macro')
#     },
#     # 'SVM': {
#     #     'accuracy': accuracy_score(y_test, y_pred_svm),
#     #     # 'precision': precision_score(y_test, y_pred_svm, average='macro'),
#     #     # 'recall': recall_score(y_test, y_pred_svm, average='macro'),
#     #     # 'f1_score': f1_score(y_test, y_pred_svm, average='macro')
#     # }
# }
# print(metrics)


without br
0,9056936612441322;0,8929665604465888;0,9046912327310996;0,8986755491302927

without km
0,8933379587257536;0,8795907081750391;0,8898862629568614;0,8846252811730392

without sentido_via
0,9083581825160166;0,9012686532593571;0,9073114949105466;0,9042523026084669

without condicao_metereologica
0,9092955625756547;0,9001600904300741;0,9084998340537614;0,9042359484390906

without tipo_pista
0,9071772312597798;0,899098317543478;0,9061375742881816;0,9025636367412472

without tipo_veiculo
0,8962608130849399;0,8914369116950105;0,8995110415985835;0,8954088002315603

without ano_fabricacao_veiculo
0,9068819934457205;0,8995200621099685;0,9032287748383128;0,9013536749357596

without tipo_envolvido
0,9067269935933394;0,8993156883518244;0,9053870208163439;0,9023075585272496

without idade
0,8851082046588528;0,8716832449278551;0,8739435934487172;0,8728069280154116

without sexo
0,9074134215110271;0,9001069009888338;0,9060530700888854;0,9030390994243768

0,9094136577012784;0,901932518474354;

# APRIORI - Regra de associação

In [46]:
features = ['br', 'km', 'causa_acidente', 'tipo_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tipo_veiculo', 'ano_fabricacao_veiculo', 'tipo_envolvido', 'idade', 'sexo']

X = data_interesse_le[features]#[:limit]
y = data_interesse_le['estado_fisico']#[:limit]


In [142]:
class Column:
    def __init__(self, column: str, value: str, index: int):
        self.column = column
        self.value = value
        self.index = index
    
    def __str__(self):
        return f'{self.column}:{self.value}'

rows_apriori = [None] * len(data)

for column in data[features]:
    for index, value in enumerate(data[column]):
        if not rows_apriori[index]:
            rows_apriori[index] = []
        rows_apriori[index].append(Column(column, value, index))

In [166]:
import hashlib

associations_hash_value = {}

class ItemSet:
    def __init__(self, pre: [Column], pos: [Column], regra: [Column]):
        self.pre = pre
        self.pos = pos
        self.regra = regra

        self.suporte_pre = None
        self.suporte_pos = None
        self.suporte_regra = None

        self.confiaca = None
        self.lift = None
    
    def __str__(self):
        return f"{[str(x) for x in self.pre]} -> {[str(x) for x in self.pos]}, sup_pre:{self.suporte_pre * 100}%,  sup_pos:{self.suporte_pos * 100}%,  sup_regra:{self.suporte_regra * 100}%, conf:{self.confiaca * 100}%, lift:{self.lift}"

    def to_csv(self):
        return f"{[str(x) for x in self.pre]}, {[str(x) for x in self.pos]}, {self.suporte_pre}, {self.suporte_pos}, {self.suporte_regra}, {self.confiaca}, {self.lift}"

def sha3(value: any) -> str:
    return hashlib.sha3_224(str(value).encode()).hexdigest()

def generate_item_set(row: [Column], index):
    try:
        target = Column('estado_fisico', data['estado_fisico'][index], -1)
    except:
        return
    for column in row:
        regra = [column, target]
        key = sha3([x.value for x in regra])

        try :
            if associations_hash_value[key]:
                continue
        except:
            associations_hash_value[key] = ItemSet(pre=[column], pos=[target], regra=regra)



In [167]:
for index, row in enumerate(rows_apriori):
    generate_item_set(row, index)

associations: [ItemSet] = list(associations_hash_value.values())

In [168]:
calculated_associoation_ocorrences = {}
for row in rows_apriori:
    for column in row:
        key = f"{column.column}#{column.value}"
        try:
            calculated_associoation_ocorrences[key] += 1 
        except:
            calculated_associoation_ocorrences[key] = 1

for column in data['estado_fisico']:
    key = f"estado_fisico#{column}"
    try:
        calculated_associoation_ocorrences[key] += 1 
    except:
        calculated_associoation_ocorrences[key] = 1

def calculate_ocorrences(items: [ItemSet]) -> float:
    key = f"{items[0].column}#{items[0].value}"

    return calculated_associoation_ocorrences[key] / len(rows_apriori)

def calc_suporte(item: ItemSet):
    item.suporte_pre = round(calculate_ocorrences(item.pre), 6)
    item.suporte_pos = round(calculate_ocorrences(item.pos), 6)
    item.suporte_regra = round(calculate_ocorrences(item.regra), 6)
    
    item.confiaca = round(item.suporte_regra / item.suporte_pre, 6)
    item.lift = round(item.confiaca / item.suporte_pos, 6)

for assoc in associations:
    calc_suporte(assoc)

In [169]:
total = {
    'suporte_pre': [],
    'suporte_pos': [],
    'suporte_regra': [],
    'confiaca': [],
    'lift': [],
} 

for assoc in associations:
    total['suporte_pre'].append(assoc.suporte_pre)
    total['suporte_pos'].append(assoc.suporte_pos)
    total['suporte_regra'].append(assoc.suporte_regra)
    total['confiaca'].append(assoc.confiaca)
    total['lift'].append(assoc.lift)


In [172]:
for assoc in associations[:10]:
    print(assoc.to_csv())

['br:116.0'] -> ['estado_fisico:Lesões Leves'], 0.158614, 0.364647, 0.158614, 1.0, 2.742378
['km:587.0'] -> ['estado_fisico:Lesões Leves'], 0.00074, 0.364647, 0.00074, 1.0, 2.742378
['causa_acidente:Ingestão de álcool pelo condutor'] -> ['estado_fisico:Lesões Leves'], 0.048108, 0.364647, 0.048108, 1.0, 2.742378
['tipo_acidente:Queda de ocupante de veículo'] -> ['estado_fisico:Lesões Leves'], 0.119893, 0.364647, 0.119893, 1.0, 2.742378
['fase_dia:Plena Noite'] -> ['estado_fisico:Lesões Leves'], 0.336745, 0.364647, 0.336745, 1.0, 2.742378
['sentido_via:Crescente'] -> ['estado_fisico:Lesões Leves'], 0.543396, 0.364647, 0.543396, 1.0, 2.742378
['condicao_metereologica:Nublado'] -> ['estado_fisico:Lesões Leves'], 0.144708, 0.364647, 0.144708, 1.0, 2.742378
['tipo_pista:Simples'] -> ['estado_fisico:Lesões Leves'], 0.538482, 0.364647, 0.538482, 1.0, 2.742378
['tipo_veiculo:Motocicleta'] -> ['estado_fisico:Lesões Leves'], 0.205475, 0.364647, 0.205475, 1.0, 2.742378
['ano_fabricacao_veiculo:201

In [186]:
assocs_lift = []

for i, x in enumerate(associations):
    if x.suporte_regra > 0.3:
        assocs_lift.append([i, x])

In [190]:
for assoc in assocs_lift:
    print(assoc[0], '-', assoc[1])

4 - ['fase_dia:Plena Noite'] -> ['estado_fisico:Lesões Leves'], sup_pre:33.6745%,  sup_pos:36.4647%,  sup_regra:33.6745%, conf:100.0%, lift:2.742378
5 - ['sentido_via:Crescente'] -> ['estado_fisico:Lesões Leves'], sup_pre:54.3396%,  sup_pos:36.4647%,  sup_regra:54.3396%, conf:100.0%, lift:2.742378
7 - ['tipo_pista:Simples'] -> ['estado_fisico:Lesões Leves'], sup_pre:53.8482%,  sup_pos:36.4647%,  sup_regra:53.8482%, conf:100.0%, lift:2.742378
10 - ['tipo_envolvido:Condutor'] -> ['estado_fisico:Lesões Leves'], sup_pre:66.6833%,  sup_pos:36.4647%,  sup_regra:66.6833%, conf:100.0%, lift:2.742378
12 - ['sexo:Masculino'] -> ['estado_fisico:Lesões Leves'], sup_pre:74.32839999999999%,  sup_pos:36.4647%,  sup_regra:74.32839999999999%, conf:100.0%, lift:2.742378
19 - ['fase_dia:Pleno dia'] -> ['estado_fisico:Lesões Leves'], sup_pre:55.1651%,  sup_pos:36.4647%,  sup_regra:55.1651%, conf:100.0%, lift:2.742378
20 - ['sentido_via:Decrescente'] -> ['estado_fisico:Lesões Leves'], sup_pre:45.6604%,  su

In [207]:

column = associations[237]
for assoc in associations:
    if assoc.pre[0].column == column.pre[0].column and assoc.pre[0].value == column.pre[0].value:
        print(assoc)

['sentido_via:Decrescente'] -> ['estado_fisico:Lesões Leves'], sup_pre:45.6604%,  sup_pos:36.4647%,  sup_regra:45.6604%, conf:100.0%, lift:2.742378
['sentido_via:Decrescente'] -> ['estado_fisico:Lesões Graves'], sup_pre:45.6604%,  sup_pos:13.112599999999999%,  sup_regra:45.6604%, conf:100.0%, lift:7.626253
['sentido_via:Decrescente'] -> ['estado_fisico:Ileso'], sup_pre:45.6604%,  sup_pos:45.0271%,  sup_regra:45.6604%, conf:100.0%, lift:2.220885
['sentido_via:Decrescente'] -> ['estado_fisico:Óbito'], sup_pre:45.6604%,  sup_pos:5.3956%,  sup_regra:45.6604%, conf:100.0%, lift:18.53362


3097

In [185]:
f = open("assoc_v1.csv", "w")
f.write("pre, pos, sup_pre, sup_pos, sup_regra, conf, lift\n")
for assoc in associations:
    f.write(f"{assoc.to_csv()}\n")
f.close()

In [176]:
str(associations[0])

"['br:116.0'] -> ['estado_fisico:Lesões Leves'], sup_pre:15.8614%,  sup_pos:36.4647%,  sup_regra:15.8614%, conf:100.0%, lift:2.742378"